In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [2]:
data = pd.read_csv('savant_data.csv')

In [3]:
data['velocity'].mean()

np.float64(87.48987314085738)

In [4]:
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'pitch_percent'
# - 'swing_miss_percent'
# - 'arm_angle'
# - 'xbadiff'
# - 'xobp'
# - 'xslg'
# - 'pitcher_run_value_per_100'
# - 'batter_run_value_per_100'

# We will also select 'swing_miss_percent' as our initial target variable

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'pitch_percent', 'swing_miss_percent', 'arm_angle', 
    'xbadiff', 'xobp', 'xslg', 'pitcher_run_value_per_100', 'batter_run_value_per_100'
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['swing_miss_percent']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(0.0754832961748642)

In [5]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)

      Actual  Predicted
538     38.7     38.722
1822    26.6     26.595
534     41.0     40.991
2119    20.8     20.800
561     50.0     50.000
...      ...        ...
744     39.2     39.204
733     30.6     30.600
2647    21.1     21.100
4149     9.7      9.690
3819    21.7     21.700

[915 rows x 2 columns]


In [6]:
#mean_whiff = data['swing_miss_percent'].mean()
#data['Stuff+'] = (data['swing_miss_percent'] / mean_whiff) * 100

# Normalizing the columns
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = (1 - (data['xwoba'] / data['xwoba'].max())) * 100 # Inverse
data['RunExp_norm'] = (1 - (data['run_exp'] / data['run_exp'].max())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()


,swing_miss_percent,xobp,pitcher_run_value_per_100,Stuff+
0,44.9,0.234,1.495762,189.707922
1,25.7,0.311,1.273859,172.284705
2,39.5,0.261,1.302039,171.132300
3,42.3,0.231,1.112724,162.073029
4,26.0,0.285,1.117130,156.173976


In [7]:
# Sort the DataFrame by the 'Stuff+' column in descending order (higher Stuff+ is better)
features_sorted = data.sort_values(by='Stuff+', ascending=False)
features_sorted.to_csv('savant_data.csv', index=False)

In [8]:
'''
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse
data['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()
'''

"\ndata['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100\ndata['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse\ndata['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse\n\n# Assigning weights\nw1, w2, w3 = .3, .2, .5\n\n# Calculating Stuff+\ndata['Stuff+'] = (\n    w1 * data['SwingMiss_norm'] +\n    w2 * data['xwOBA_norm'] +\n    w3 * data['RunExp_norm']\n)\n\n# Display the first few rows of the Stuff+ calculation\ndata[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()\n"

In [9]:
print(data['xwoba'].min())
print(data['xwoba'].max())
print(data['xwoba'].mean())
print(data['pitcher_run_exp'].min())
print(data['pitcher_run_exp'].max())
print(data['pitcher_run_exp'].mean())


0.0
1.989
0.32474518810148734
-53.1
79.3
0.25581802274715626


In [10]:
# different normailzation so all are around 60-150, average at 100
data['pre_new_all_+'] = data['pitcher_run_exp'] - data['pitcher_run_exp'].min()
print(data['pre_new_all_+'].mean())

53.35581802274716


In [11]:
std_dev_pre = data['pitcher_run_exp'].std()
std_dev_xwoba = data['xwoba'].std()
std_dev_swm = data['swing_miss_percent'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['xwOBA_norm'] = (((data['xwoba'].mean() - data['xwoba'])) / std_dev_xwoba) * 10
data['RunExp_norm'] = ((data['pitcher_run_exp'] - data['pitcher_run_exp'].mean()) / std_dev_pre) * 5

# Assigning weights
w1, w2, w3 = .2, .3, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

data['Stuff+'] = (data['Stuff+'] + 100)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()

,swing_miss_percent,xobp,pitcher_run_value_per_100,Stuff+
0,44.9,0.234,1.495762,127.548710
1,25.7,0.311,1.273859,124.233678
2,39.5,0.261,1.302039,122.356066
3,42.3,0.231,1.112724,119.402701
4,26.0,0.285,1.117130,119.100327


In [12]:
features_sorted = data.sort_values(by='Stuff+', ascending=False)
print(data['Stuff+'].mean())
features_sorted.to_csv('savant_data.csv', index=False)

100.0


In [13]:
filtered_data = data[data['pitches'] >= 200]
filtered_data = filtered_data.sort_values(by='Stuff+', ascending=False)
filtered_data.to_csv('STUFF+Initial.csv', index=False)

In [14]:
print(filtered_data[filtered_data['pitch_type'] == 'FC'].head())

    pitches  player_id         player_name  total_pitches pitch_type  \
1      6224     669203      Burnes, Corbin          11943         FC   
17     1278     621237      Alvarado, José           3720         FC   
20     2802     661403     Clase, Emmanuel           4080         FC   
39      318     596112  Stephenson, Robert           2444         FC   
61     2028     502085    Robertson, David           3546         FC   

    pitch_percent     ba    iso  babip    slg  ...  xobpdiff  xslgdiff  \
1            52.1  0.227  0.115  0.278  0.341  ...    -0.012    -0.022   
17           34.4  0.136  0.063  0.327  0.199  ...     0.018     0.035   
20           68.7  0.212  0.071  0.259  0.283  ...    -0.022    -0.040   
39           13.0  0.101  0.152  0.143  0.253  ...    -0.020     0.008   
61           57.2  0.196  0.084  0.283  0.280  ...    -0.005    -0.041   

    wobadiff  swing_miss_percent  arm_angle      Stuff+  SwingMiss_norm  \
1     -0.015                25.7       41.2  12

In [15]:
std_dev_ba = data['ba'].std()
std_dev_ls = data['launch_speed'].std()
std_dev_swm = data['swing_miss_percent'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['ls_norm'] = ((data['launch_speed'].mean() - data['launch_speed']) / std_dev_ls) * 10
data['ba_norm'] = ((data['ba'].mean() - data['ba']) / std_dev_ba) * 5

# Assigning weights
w1, w2, w3 = 1/3, 1/3, 1/3

# Calculating Stuff+
data['Unhittable+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['ls_norm'] +
    w3 * data['ba_norm']
)

data['Unhittable+'] = (data['Unhittable+'] + 100)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'ba', 'launch_speed', 'Unhittable+']].head()

,swing_miss_percent,ba,launch_speed,Unhittable+
0,44.9,0.169,86.3,108.181238
1,25.7,0.227,87.8,100.603362
2,39.5,0.169,85.6,107.052060
3,42.3,0.174,84.6,108.541903
4,26.0,0.211,91.3,98.411275


In [16]:
filtered_data_unhit = data[data['pitches'] >= 200]
filtered_data_unhit = filtered_data_unhit.sort_values(by='Unhittable+', ascending=False)
filtered_data_unhit.to_csv('Unhittable+Initial.csv', index=False)
data['Unhittable+'].mean()

np.float64(100.0)

In [107]:
# Stuff Model!
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'velocity'
# - 'api_break_z_induced'
# - 'api_break_x_arm'
# - 'release_extension'
# - 'spin_rate'
# - 'release_pos_x'
# - 'release_pos_z'
# - 'x_diff'  
# - 'z_diff'
# - 'Stuff+'

# Adding New Variables, finding primary fastball
#if (
data['release_pos_x_norm'] = data['release_pos_x']
data['release_pos_x_norm'] = data['release_pos_x_norm'].apply(lambda x: -x if x < 0 else x)

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'velocity', 'api_break_z_induced', 'api_break_x_arm', 'release_extension', 'spin_rate', 'release_pos_x_norm', 'release_pos_z', 'x_diff', 'z_diff'  
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['Stuff+']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(23.364712764815533)

In [108]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)
print(y_test.max())
print(y_pred.max())
print(y_test.mean())
print(y_pred.mean())
print(y_test.min())
print(y_pred.min())

          Actual   Predicted
3232   97.492353   99.885158
692   104.951778   99.984860
1259  102.810011  100.130473
497   105.928896  101.898679
549   105.661751  102.191947
...          ...         ...
4354   92.081564   97.180854
334   107.143383  103.049149
3156   97.689569  104.989850
921   103.993751   99.472701
679   105.009200  100.112090

[881 rows x 2 columns]
117.47678452132428
108.29931102093965
100.24486546163781
100.00213898497891
57.49178720691503
90.26473920244052


In [114]:
my_slider = [82, -.08333, -.91667, 6.5, 2800, 2.55, 6.02, 0, 0, False, False, False, False, False, False, False, False, False, False, True, False, False]
#a_slider = rf_model.predict([my_slider], columns=features_encoded)

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")
a_slider = rf_model.predict([my_slider])
print(a_slider)

[98.58621968]


In [115]:
my_fastball = [92, 1.4, 1.2, 6.5, 2300, 2.55, 6.02, 0, 0, False, False, False, True, False, False, False, False, False, False, False, False, False]
#a_slider = rf_model.predict([my_slider], columns=features_encoded)

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")
a_fastball = rf_model.predict([my_fastball])
print(a_fastball)

[97.22943488]


In [116]:
# Importing XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Initialize the XGBoost model
xgb_model = XGBRegressor(
    random_state=42,  # Ensure reproducibility
    n_estimators=300,  # Number of trees (default: 100)
    learning_rate=0.01,  # Step size shrinkage (default: 0.3)
    max_depth=4,       # Maximum tree depth (default: 6)
    objective='reg:squarederror'  # Regression objective
)

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 23.423670872441974


In [117]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(y_test.max())
print(y_pred.max())
print(y_test.mean())
print(y_pred.mean())
print(y_test.min())
print(y_pred.min())

117.47678452132428
104.93587
100.24486546163781
99.96469
57.49178720691503
91.662056


In [118]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4, 6, 8]
}

# Initialize the model
xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')

# Perform Grid Search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

best_xgb_model = grid_search.best_estimator_


Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_es

In [119]:
print(data.head())

   pitches  player_id      player_name  total_pitches pitch_type  \
0     4861     656302     Cease, Dylan          12527         SL   
1     6224     669203   Burnes, Corbin          11943         FC   
2     4610     450203  Morton, Charlie          11523         CU   
3     4189     592332   Gausman, Kevin          11722         FS   
4     5534     543037     Cole, Gerrit          11058         FF   

   pitch_percent     ba    iso  babip    slg  ...  RunExp_norm  pre_new_all_+  \
0           38.8  0.169  0.098  0.262  0.267  ...    43.149478          125.8   
1           52.1  0.227  0.115  0.278  0.341  ...    47.080595          132.4   
2           40.0  0.169  0.098  0.267  0.267  ...    35.585056          113.1   
3           35.7  0.174  0.096  0.287  0.270  ...    27.603697           99.7   
4           50.0  0.211  0.168  0.277  0.379  ...    36.657179          114.9   

    ls_norm   ba_norm  Unhittable+  release_pos_x_norm  fastball_x  \
0  3.306550  3.414644   108.181238